In [1]:
import pandas as pd
import spacy
from spacy import displacy

In [2]:
df = pd.read_csv("data/lyrics/genius_v2.csv")

In [3]:
genius=df.iloc[:,2:]

In [4]:
nlp = spacy.load("en_core_web_lg")


In [5]:
neighborhoods=pd.read_csv("data/neighborhoods/cleaned_neighborhoods.csv").iloc[:,1:].Neighborhoods.values.sum().split(", ")

In [6]:
import regex as re

In [7]:
start_template="\[\"(.*) Lyrics\["
m=re.search(start_template,'"["Unforgettable Lyrics[Chorus: Swae Lee]')
m.group(1)

'Unforgettable'

In [8]:
for ent in nlp("Feelin' like I'm fresh out of Bronx").ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


Feelin 0 6 ORG
Bronx 30 35 GPE


In [9]:
LoI=[]
for artist,songs in zip(genius.Name, genius.lyrics):
    lines=str(songs).split("\\n")
    for i,line in enumerate(str(songs).split("\\n")):
        m=re.search(start_template,line)

        if m :
            song_name = m.group(1)
        else:
            doc = nlp(line)
            for ent in doc.ents:
                if ent.label_=="GPE" or ent.label_=="LOC" or ent.text in neighborhoods:
                    LoI.append([artist, song_name, ent.text,line, lines[i-1] if i!=0 else "", lines[i+1] if i<(len(lines)-1) else ""])


In [10]:
df=pd.DataFrame(LoI, columns=["Artist", "Song Name", "GPE", "Line", "Line Before", "Line After"])
df.to_csv("data/lyrics/genius_ner_v2.csv")

In [11]:
df.GPE.value_counts()

New York           94
Brooklyn           56
Montana            35
Earth              26
Harlem             25
                   ..
Clinton             1
Brooklyn Beirut     1
Gotham City         1
Beverly             1
Shabazz             1
Name: GPE, Length: 480, dtype: int64

In [12]:
df.shape

(1135, 6)

In [13]:
view =df[df.GPE=="Brooklyn"]
view.shape

(56, 6)

In [14]:
view.iloc[:30].apply(lambda row : displacy.render(nlp(row["Line Before"]+"\n"+row["Line"]+"\n"+row["Line After"]),style="ent"),axis=1)

86     None
258    None
303    None
417    None
420    None
421    None
462    None
504    None
505    None
541    None
550    None
563    None
588    None
612    None
682    None
778    None
808    None
812    None
813    None
814    None
815    None
845    None
846    None
847    None
848    None
850    None
851    None
852    None
883    None
913    None
dtype: object

In [13]:
view = df["Harlem" == df.GPE]

In [15]:
doc = nlp("""I'll leave you lost, mount you on a cross
Whip you like a horse, sacrifice your life to a higher force
Then I'll stomp your corpse
It's the Bronx of course, recognize the accent""")

In [16]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Bronx GPE


In [17]:
#displacy.render(doc, style="ent")